In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('airline-price-classification.csv')

In [3]:
lbl_enc = LabelEncoder()
df['TicketCategory'] = lbl_enc.fit_transform(df[["TicketCategory"]])
print(df['TicketCategory'])

0         3
1         3
2         0
3         2
4         2
         ..
240203    0
240204    1
240205    2
240206    2
240207    2
Name: TicketCategory, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
df["date"]=pd.to_datetime(df["date"])
df["date"]=df["date"].dt.strftime("%m/%d/%Y")
pd.DatetimeIndex(df["date"]).weekday
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['dayofyear'] = pd.DatetimeIndex(df['date']).dayofyear

In [5]:
df['ch_code'] = lbl_enc.fit_transform(df[["ch_code"]])
print(df['ch_code'])

0         7
1         7
2         3
3         3
4         7
         ..
240203    2
240204    2
240205    1
240206    7
240207    6
Name: ch_code, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
df["dep_time"]=pd.to_datetime(df["dep_time"])
df['dep_time'] = df['dep_time'].dt.strftime("%-H:%M")
df["dep_hour"]=pd.DatetimeIndex(df["dep_time"]).hour
df["dep_minute"]=pd.DatetimeIndex(df["dep_time"]).minute

In [7]:
df["hours_taken"] = df["time_taken"].str.split('h').str.get(0)
df["minutes_taken"] = df["time_taken"].str[4:6]
df["minutes_taken"] = df["minutes_taken"].str.replace('m', '')
df["minutes_taken"] = df["minutes_taken"].str.replace('h', '')
df["hours_taken"] = pd.to_numeric(df["hours_taken"])
df["minutes_taken"] = pd.to_numeric(df["minutes_taken"], errors='coerce')
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,TicketCategory,month,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...",3,5,3,123,9,45,10.0,10.0
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}",3,3,18,77,5,45,6.0,30.0
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",0,9,3,246,14,30,8.0,10.0
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}",2,3,15,74,10,40,9.0,40.0
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}",2,3,22,81,12,30,7.0,25.0


In [8]:
df["stop"] = df["stop"].str.split('-').str.get(0)
df["stop"] = df["stop"].replace(['non'], 0)
df.isna().sum() #  28944 null vals
df["stop"] = df["stop"].replace(['2+'], 2) # Indicates for 2 or more stops
df['stop'] = df['stop'].fillna(0)
df['stop'] = pd.to_numeric(df['stop'])
# print(df[9:14])

In [9]:
df["arr_time"]=pd.to_datetime(df["arr_time"])
df['arr_time'] = df['arr_time'].dt.strftime("%-H:%M")
df["arr_hour"]=pd.DatetimeIndex(df["arr_time"]).hour
df["arr_minute"]=pd.DatetimeIndex(df["arr_time"]).minute
df["arr_hour"] = pd.to_numeric(df["arr_hour"])
df["arr_minute"] = pd.to_numeric(df["arr_minute"])
df.head()


,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,TicketCategory,month,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken,arr_hour,arr_minute
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...",3,5,3,123,9,45,10.0,10.0,19,55
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}",3,3,18,77,5,45,6.0,30.0,12,15
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",0,9,3,246,14,30,8.0,10.0,22,40
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}",2,3,15,74,10,40,9.0,40.0,20,20
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}",2,3,22,81,12,30,7.0,25.0,19,55


In [10]:
df['type'] = lbl_enc.fit_transform(df[["type"]])
print(df['type'])

0         0
1         0
2         1
3         1
4         1
         ..
240203    1
240204    0
240205    1
240206    1
240207    1
Name: type, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
df['source'] = df['route'].str.split( ', ').str.get(0).str.split(':').str.get(1)
df['destination'] = df['route'].str.split( ', ').str.get(1).str.split(':').str.get(1).str.split('}').str.get(0)
df['source'] = df['source'].str.replace('\'', "")
df['destination'] = df['destination'].str.replace('\'', "")
df['source'] = lbl_enc.fit_transform(df[["source"]])
print(df['source'])
df['destination'] = lbl_enc.fit_transform(df[["destination"]])
print(df['destination'])
df = df.dropna()
df = df.drop(['airline', 'date', 'dep_time', "time_taken", 'arr_time', 'route',], axis=1)
#df = pd.get_dummies(df)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0         0
1         2
2         5
3         5
4         1
         ..
240203    1
240204    2
240205    2
240206    2
240207    4
Name: source, Length: 240208, dtype: int64
0         3
1         3
2         3
3         2
4         4
         ..
240203    3
240204    4
240205    0
240206    5
240207    3
Name: destination, Length: 240208, dtype: int64


In [12]:
X = df.loc[:, df.columns != 'TicketCategory']
Y = df['TicketCategory']
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=8)

In [13]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
scaler = ss.fit(XTrain)
trainX_scaled = scaler.transform(XTrain)
testX_scaled = scaler.transform(XTest)

In [14]:
from keras.layers import Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [22]:
y_train = to_categorical(YTrain)
y_test = to_categorical(YTest)

In [23]:
# define model
model = Sequential()
model.add(Dense(256, input_dim=XTrain.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [24]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])              

In [27]:
history = model.fit(XTrain, y_train, validation_data=(XTest, y_test), epochs=100, verbose=1)


Epoch 1/100
6006/6006 [==============================] - 13s 2ms/step - loss: 0.5541 - accuracy: 0.7123 - val_loss: 0.5432 - val_accuracy: 0.7161
Epoch 2/100
6006/6006 [==============================] - 12s 2ms/step - loss: 0.5426 - accuracy: 0.7200 - val_loss: 0.5419 - val_accuracy: 0.7251
Epoch 3/100
6006/6006 [==============================] - 13s 2ms/step - loss: 0.5334 - accuracy: 0.7269 - val_loss: 0.5350 - val_accuracy: 0.7190
Epoch 4/100
6006/6006 [==============================] - 15s 2ms/step - loss: 0.5274 - accuracy: 0.7321 - val_loss: 0.5315 - val_accuracy: 0.7194
Epoch 5/100
6006/6006 [==============================] - 12s 2ms/step - loss: 0.5185 - accuracy: 0.7399 - val_loss: 0.5176 - val_accuracy: 0.7404
Epoch 6/100
6006/6006 [==============================] - 12s 2ms/step - loss: 0.5050 - accuracy: 0.7510 - val_loss: 0.5084 - val_accuracy: 0.7421
Epoch 7/100
6006/6006 [==============================] - 13s 2ms/step - loss: 0.4977 - accuracy: 0.7560 - val_loss: 0.5088 -

In [28]:
train_acc = model.evaluate(XTrain, y_train, verbose=1)
test_acc = model.evaluate(XTest, y_test, verbose=1)

1502/1502 [==============================] - 2s 1ms/step - loss: 0.3600 - accuracy: 0.8445


In [29]:
model.summary()
print(train_acc)
print(test_acc)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               4096      
                                                                 
 dense_7 (Dense)             (None, 64)                16448     
                                                                 
 dense_8 (Dense)             (None, 4)                 260       
                                                                 
Total params: 20,804
Trainable params: 20,804
Non-trainable params: 0
_________________________________________________________________
[0.3501461446285248, 0.8465997576713562]
[0.3599664568901062, 0.8445318937301636]
